In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [3]:
device = 'cpu'
dataset = IncomeDataset(device=device)

In [4]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[800,800,800,800,800,800,800,800,800,800])
        # model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[200,200,200,200,200,200,100])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[800,800,800,800,800,800,800,800,800,800])
        # model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[200,200,200,200,200,200,100])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[800,800,800,800,800,800,800,800,800,800])
        # model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[200,200,200,200,200,200,100])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds):
    
    SGD_train, _, SGD_test = lr_kde_model_runner(dataset, SGD_hp, seeds)
    EI_fc_train, _, EI_fc_test = lr_fc_model_runner(dataset, EI_hp_fc, seeds)
    EI_kde_train, _, EI_kde_test = lr_kde_model_runner(dataset, EI_hp_kde, seeds)
    EI_fb_train, _, EI_fb_test = lr_fb_model_runner(dataset, EI_hp_fb, seeds)
    
    return SGD_train, EI_fc_train, EI_kde_train, EI_fb_train, SGD_test, EI_fc_test, EI_kde_test, EI_fb_test

def fb_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def kde_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def fc_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fc_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [5]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 50
SGD_hp_test['batch_size'] = 256
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 1.1
SGD_hp_test['effort_iter'] = 20
SGD_hp_test['effort_lr'] = 15

kde_hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 50/50 [02:54<00:00,  3.49s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [02:56<00:00,  3.53s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [03:02<00:00,  3.65s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [02:59<00:00,  3.59s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
         0.0001          0          0.909375         0.75625  0.037037   0.11962    0.0927593  0.131516
         0.001           0          0.996875         0.7625   0.0425457  0.0505538  0.0516634  0.0516634
         0.01            0          0.946875         0.7875   0.040293   0.0894778  0.111155   0.111155
         0.1             0          0.407813         0.45625  0          0          0          0


The learning rate is decided as 0.01.


In [6]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.0001
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [7]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.0001]
EI_hp_test['lambda_'] = [0, 0.2, 0.3, 0.4, 0.5]
EI_hp_test['fairness'] = 'EI'

kde_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training:   0%|          | 0/50 [00:00<?, ?epochs/s]

Training: 100%|██████████| 50/50 [05:16<00:00,  6.34s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:17<00:00,  6.34s/epochs]


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:15<00:00,  6.32s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:17<00:00,  6.36s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
         0.0001        0            0.909375         0.75625  0.037037    0.11962    0.0927593  0.131516
         0.0001        0.2          0.910937         0.76875  0.00931966  0.10712    0.0927593  0.107993
         0.0001        0.3          0.920312         0.78125  0.0501166   0.0821203  0.0790607  0.0790607
         0.0001        0.4          0.920312         0.78125  0.0576923   0.0947785  0.107632   0.107632
         0.0001        0.5          0.929688         0.775    0.0621742   0.0758703  0.065362   0.0727078


The lambda for EI is decided as 0.4.

In [8]:
EI_hp_kde = EI_hp_test.copy()
EI_hp_kde['learning_rate'] = 0.0001
EI_hp_kde['lambda_'] = 0.3

In [9]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.0001]
EI_hp_test['lambda_'] = [0, 0.6, 0.8, 0.9, 0.95, 0.99]
EI_hp_test['fairness'] = 'EI'

fc_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training:   0%|          | 0/50 [00:00<?, ?epochs/s]

Training: 100%|██████████| 50/50 [05:13<00:00,  6.27s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:20<00:00,  6.42s/epochs]


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:52<00:00,  7.06s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [06:08<00:00,  7.37s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
         0.0001       0             0.909375         0.75625  0.037037   0.11962    0.0927593  0.131516
         0.0001       0.6           0.909375         0.775    0.0632593  0.101028   0.107632   0.107632
         0.0001       0.8           0.8875           0.76875  0.075      0.0816456  0.0778865  0.0778865
         0.0001       0.9           0.592187         0.54375  0          0          0          0
         0.0001       0.95          0.592187         0.54375  0          0          0          0
         0.0001       0.99          0.592187         0.54375  0          0          0          0


In [10]:
EI_hp_fc = EI_hp_test.copy()
EI_hp_fc['learning_rate'] = 0.0001
EI_hp_fc['lambda_'] = 0.95

In [11]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.0001]
EI_hp_test['lambda_'] = [0, 0.1, 0.2, 0.3, 0.4]
EI_hp_test['fairness'] = 'EI'

fb_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training:   0%|          | 0/50 [00:00<?, ?epochs/s]

Training: 100%|██████████| 50/50 [05:33<00:00,  6.67s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:34<00:00,  6.70s/epochs]


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:52<00:00,  7.05s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [05:40<00:00,  6.80s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
         0.0001        0            0.909375         0.75625  0.037037   0.11962    0.0927593  0.131516
         0.0001        0.1          0.923438         0.775    0.0274094  0.0638449  0.0962818  0.0962818
         0.0001        0.2          0.904687         0.75     0.0450281  0.0641614  0.0700587  0.0700587
         0.0001        0.3          0.85625          0.75625  0.0133333  0.0712025  0.072407   0.072407
         0.0001        0.4          0.817187         0.675    0.025641   0.0662184  0.0669276  0.0669276


In [12]:
EI_hp_fb = EI_hp_test.copy()
EI_hp_fb['learning_rate'] = 0.0001
EI_hp_fb['lambda_'] = 0.3

#### Model training

In [13]:
seeds = np.arange(1,6)

SGD_train, EI_fc_train, EI_kde_train, EI_fb_train, SGD_test, EI_fc_test, EI_kde_test, EI_fb_test = experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds)

/var/folders/rl/z562j0n16b75w5wm19c5jq7c0000gn/T/ipykernel_36159/3716580694.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training:   0%|          | 0/50 [00:00<?, ?epochs/s]

Training: 100%|██████████| 50/50 [02:59<00:00,  3.59s/epochs]


training seed 2 started


...

training seed 4 started


Training: 100%|██████████| 50/50 [05:19<00:00,  6.38s/epochs]


training seed 5 started


Training: 100%|██████████| 50/50 [05:22<00:00,  6.44s/epochs]


Training finished for all seeds.


In [14]:
result = []
models = ["SGD train", "EI FC train", "EI KDE train", "EI FB train","SGD test", "EI FC test", "EI KDE test", "EI FB test"]
sol = [SGD_train, EI_fc_train, EI_kde_train, EI_fb_train, SGD_test, EI_fc_test, EI_kde_test, EI_fb_test]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model           accuracy_mean    accuracy_var     ei_mean      ei_var    dp_mean      dp_var    eo_mean     eo_var    eodd_mean    eodd_var
------------  ---------------  --------------  ----------  ----------  ---------  ----------  ---------  ---------  -----------  ----------
SGD train            0.921562      0.0121594   0.00964491  0.00508963  0.043578   0.0146729   0.0276663  0.0217822    0.0276663   0.0217822
EI FC train          0.585313      0.00756482  0           0           0          0           0          0            0           0
EI KDE train         0.906563      0.0138491   0.0164421   0.0139024   0.0384757  0.00733642  0.0164138  0.016483     0.0175498   0.0155312
EI FB train          0.874375      0.0153603   0.00826157  0.00649534  0.0371103  0.00732055  0.0369577  0.0149186    0.0369577   0.0149186
SGD test             0.779         0.008       0.0542749   0.0085961   0.03825    0.0261226   0.0807317  0.0500574    0.0808972   0.049884
EI FC test           0.59    